## NN in Pytorch
Defining different metrics that define a [candle stick](https://stock-market-forecast.blogspot.com/2012/02/automated-recognition-of-candlestick.html).  

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plot
import time
import math
import quandl

In [4]:
stock_val=quandl.get("BCHARTS/BITFINEXUSD")
RB = 100.0 * (stock_val.Close - stock_val.Open) / (stock_val.Open)
US = 100.0 * (stock_val.Close - stock_val.Open) / (stock_val.High - stock_val.Open)
LS = 100.0 * (stock_val.Close - stock_val.Open) / (stock_val.Close - stock_val.Low)
data = torch.zeros((RB.size,3))
data[:,0] = torch.from_numpy(RB.values)
data[:,1] = torch.from_numpy(US.values)
data[:,2] = torch.from_numpy(LS.values)
inputdata = torch.Tensor(data)

Define a Neurual Network model with 3 input features, 2 hidden layers and one output layer.![alt text](https://ml4a.github.io/images/figures/neural-net.png)


In [5]:
D_in, H, D_out = 3, 2, 1

In [6]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

Define RNN module with 10 previous outputs to get the next output. RNN module ![alt text](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

In [7]:
input_size, hidden_size, output_size = 100, 128, 1


In [8]:
def getInputTensor(x, index, input_size):
    inputTensor = x[0, index:(index+input_size)].unsqueeze(0)
    return inputTensor

class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input.unsqueeze(0), hidden),1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return Variable(torch.zeros(1,self.hidden_size))

Combine Two Layer NN and RNN togther for learning

In [9]:
rnn = RNN(input_size, hidden_size, output_size)
learning_rate = 0.0005
criterion = nn.MSELoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)
x = Variable(torch.Tensor(RB.values)).unsqueeze(0)
y = Variable(torch.Tensor(RB[2:len(RB)].values)).unsqueeze(0)

In [10]:
def train(input_tensor, output_tensor):
    loss = 0
    hidden = rnn.initHidden()

    for i in range(input_tensor.size()[0]):
        output, hidden = rnn(input_tensor[i], hidden)
        loss += criterion(output, output_tensor[i])
        optimizer.zero_grad()
    loss.backward()
    optimizer.step()
#     for p in rnn.parameters():
#         p.data.add_(-learning_rate, p.grad.data)

    return output, loss.data[0] / input_tensor.size()[0]

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [11]:
rnn = RNN(input_size, hidden_size, output_size)

n_iters = 1000
print_every = 100
plot_every = 100
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(0, n_iters + 1):
    output, loss = train(getInputTensor(x, iter, input_size), y[0, iter].unsqueeze(0))
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

0m 0s (0 0%) 218.3377
0m 0s (100 10%) 21.6333
0m 0s (200 20%) 59.8745
0m 0s (300 30%) 61.3775
0m 0s (400 40%) 5.8834
0m 0s (500 50%) 7.1114
0m 0s (600 60%) 20.1230
0m 0s (700 70%) 5.4103
0m 0s (800 80%) 0.3697
0m 0s (900 90%) 0.5287
0m 0s (1000 100%) 0.1403
